In [1]:
import sympy as smp
import numpy as np
import matplotlib.pyplot as plt
import scipy as sip

""" from sympy.physics.mechanics import dynamicsymbols, init_vprinting
init_vprinting """


' from sympy.physics.mechanics import dynamicsymbols, init_vprinting\ninit_vprinting '

In [2]:
# declare variables
t, l1, l2, m1, m2, w, a, g = smp.symbols(r't l_1 l_2 m_1 m_2 \omega A g')
x1, y1, theta1, x2, y2, theta2 =smp.symbols(r'x_1 y_1 \theta_1 x_2 y_2 \theta_2', cls=smp.Function)
x1, y1, theta1, x2, y2, theta2 = x1(t), y1(t), theta1(t), x2(t), y2(t), theta2(t)

# variable conversions
""" x1 = a*smp.cos(w*t)+l1*smp.sin(theta1)
y1 = - l1*smp.cos(theta1)
 """
x1_eq = smp.Eq(x1, a*smp.cos(w*t)+l1*smp.sin(theta1))
y1_eq = smp.Eq(y1, - l1*smp.cos(theta1))

x2_eq = smp.Eq(x2, x1 + l2*smp.sin(theta2))
y2_eq = smp.Eq(y2, y1 - l2*smp.cos(theta2))

x2_eq = x2_eq.subs(x1, x1_eq.rhs)
y2_eq = y2_eq.subs(y1, y1_eq.rhs)

# derivatives
theta1_dot = theta1.diff(t)
theta2_dot = theta2.diff(t)

theta1_ddot = theta1_dot.diff(t)
theta2_ddot = theta2_dot.diff(t)

x1_dot_eq = smp.Eq(x1.diff(t), x1_eq.rhs.diff(t))
y1_dot_eq = smp.Eq(y1.diff(t), y1_eq.rhs.diff(t))
x2_dot_eq = smp.Eq(x2.diff(t), x2_eq.rhs.diff(t))
y2_dot_eq = smp.Eq(y2.diff(t), y2_eq.rhs.diff(t))

# calculate the lagrangian
Kin = smp.symbols('K', cls=smp.Function)
Kin = Kin(t)
Kin_eq = smp.Eq(Kin,.5*m1*(x1_dot_eq.lhs**2+y1_dot_eq.lhs**2)+.5*m2*(x2_dot_eq.lhs**2+y2_dot_eq.lhs**2))
Kin_eq_subs = Kin_eq.subs([(x1,x1_eq.rhs), (x2,x2_eq.rhs),(y1,y1_eq.rhs), (y2,y2_eq.rhs)]).expand().simplify()

Pot = smp.symbols('P', cls=smp.Function)
Pot = Pot(t)
Pot_eq = smp.Eq(Pot,m1*g*y1+m2*g*y2)
Pot_eq_subs = Pot_eq.subs([(x1,x1_eq.rhs), (x2,x2_eq.rhs),(y1,y1_eq.rhs), (y2,y2_eq.rhs)]).expand().simplify()

Lagrange = smp.symbols(r'\mathcal{L}', cls=smp.Function)
Lagrange = Lagrange(t, theta1, theta2, theta1_dot, theta2_dot) #(t,x1, y1, theta1, x2, y2, theta2, x1_dot_eq.lhs, y1_dot_eq.lhs, theta1_dot, x2_dot_eq.lhs, y2_dot_eq.lhs, theta2_dot)
Lagrange_eq = smp.Eq(Lagrange, Kin - Pot)
Lagrange_eq_subs = Lagrange_eq.subs([(Kin, Kin_eq_subs.rhs), (Pot, Pot_eq_subs.rhs)]).expand().simplify()


In [3]:
# solve Lagrange 
L1 = smp.diff(Lagrange_eq_subs.rhs, theta1) - smp.diff(smp.diff(Lagrange_eq_subs.rhs, theta1_dot), t)
L1 = L1.expand().simplify()
L1_eq = smp.Eq(L1,0)

L2 = smp.diff(Lagrange_eq_subs.rhs, theta2) - smp.diff(smp.diff(Lagrange_eq_subs.rhs, theta2_dot), t)
L2 = L2.expand().simplify()
L2_eq = smp.Eq(L2,0)

solutions = smp.solve([L1, L2], (theta1_ddot, theta2_ddot), simplify=True, rational=False)
solution_theta1_ddot, solution_theta2_ddot = solutions[theta1_ddot], solutions[theta2_ddot]
solution_theta1_ddot, solution_theta2_ddot


((A*\omega**2*m_1*cos(\omega*t - \theta_1(t))/2 + A*\omega**2*m_1*cos(\omega*t + \theta_1(t))/2 + A*\omega**2*m_2*cos(\omega*t - \theta_1(t))/4 + A*\omega**2*m_2*cos(\omega*t + \theta_1(t))/4 - A*\omega**2*m_2*cos(\omega*t - \theta_1(t) + 2*\theta_2(t))/4 - A*\omega**2*m_2*cos(\omega*t + \theta_1(t) - 2*\theta_2(t))/4 - g*m_1*sin(\theta_1(t)) - g*m_2*sin(\theta_1(t) - 2*\theta_2(t))/2 - g*m_2*sin(\theta_1(t))/2 - l_1*m_2*sin(2*\theta_1(t) - 2*\theta_2(t))*Derivative(\theta_1(t), t)**2/2 - l_2*m_2*sin(\theta_1(t) - \theta_2(t))*Derivative(\theta_2(t), t)**2)/(l_1*(m_1 - m_2*cos(\theta_1(t) - \theta_2(t))**2 + m_2)),
 (-A*\omega**2*m_1*cos(\omega*t)*cos(\theta_1(t) - \theta_2(t))*cos(\theta_1(t)) + A*\omega**2*m_1*cos(\omega*t)*cos(\theta_2(t)) - A*\omega**2*m_2*cos(\omega*t)*cos(\theta_1(t) - \theta_2(t))*cos(\theta_1(t)) + A*\omega**2*m_2*cos(\omega*t)*cos(\theta_2(t)) + g*m_1*sin(\theta_1(t))*cos(\theta_1(t) - \theta_2(t)) - g*m_1*sin(\theta_2(t)) + g*m_2*sin(\theta_1(t))*cos(\theta_1

In [4]:
# small angle approximation
# assume solutions in form amp * cos (omega t)



In [5]:
m=smp.symbols("m")
solution_theta1_ddot.subs([(m1,1),(m2,m), (a, 1)])

(\omega**2*m*cos(\omega*t - \theta_1(t))/4 + \omega**2*m*cos(\omega*t + \theta_1(t))/4 - \omega**2*m*cos(\omega*t - \theta_1(t) + 2*\theta_2(t))/4 - \omega**2*m*cos(\omega*t + \theta_1(t) - 2*\theta_2(t))/4 + \omega**2*cos(\omega*t - \theta_1(t))/2 + \omega**2*cos(\omega*t + \theta_1(t))/2 - g*m*sin(\theta_1(t) - 2*\theta_2(t))/2 - g*m*sin(\theta_1(t))/2 - g*sin(\theta_1(t)) - l_1*m*sin(2*\theta_1(t) - 2*\theta_2(t))*Derivative(\theta_1(t), t)**2/2 - l_2*m*sin(\theta_1(t) - \theta_2(t))*Derivative(\theta_2(t), t)**2)/(l_1*(-m*cos(\theta_1(t) - \theta_2(t))**2 + m + 1))

In [6]:
solution_theta2_ddot.subs([(m1,1),(m2,m), (a, 1)])

(-\omega**2*m*cos(\omega*t)*cos(\theta_1(t) - \theta_2(t))*cos(\theta_1(t)) + \omega**2*m*cos(\omega*t)*cos(\theta_2(t)) - \omega**2*cos(\omega*t)*cos(\theta_1(t) - \theta_2(t))*cos(\theta_1(t)) + \omega**2*cos(\omega*t)*cos(\theta_2(t)) + g*m*sin(\theta_1(t))*cos(\theta_1(t) - \theta_2(t)) - g*m*sin(\theta_2(t)) + g*sin(\theta_1(t))*cos(\theta_1(t) - \theta_2(t)) - g*sin(\theta_2(t)) + l_1*m*sin(\theta_1(t) - \theta_2(t))*Derivative(\theta_1(t), t)**2 + l_1*sin(\theta_1(t) - \theta_2(t))*Derivative(\theta_1(t), t)**2 + l_2*m*sin(2*\theta_1(t) - 2*\theta_2(t))*Derivative(\theta_2(t), t)**2/2)/(l_2*(-m*cos(\theta_1(t) - \theta_2(t))**2 + m + 1))

In [7]:
vx1_num = smp.lambdify((t,a,w,l1,l2,m1,m2,theta1,theta2,theta1_dot,theta2_dot), smp.diff(x1_eq.rhs, t))
vx2_num = smp.lambdify((t,a,w,l1,l2,m1,m2,theta1,theta2,theta1_dot,theta2_dot), smp.diff(x2_eq.rhs, t))
vy1_num = smp.lambdify((t,a,w,l1,l2,m1,m2,theta1,theta2,theta1_dot,theta2_dot), smp.diff(y1_eq.rhs, t))
vy2_num = smp.lambdify((t,a,w,l1,l2,m1,m2,theta1,theta2,theta1_dot,theta2_dot), smp.diff(y2_eq.rhs, t))